Code based on https://github.com/pytorch/examples/blob/master/mnist/main.py

This exercise covers two aspects:
* In tasks 1-6 you will implement mechanisms that allow training deeper models (better initialization, batch normalization). Note that for dropout and batch norm you are expected to implement it yourself without relying on ready-made components from Pytorch.
* In task 7 you will implement a convnet using [conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html).


Tasks:
1. Check that the given implementation reaches 95% test accuracy for
   architecture input-64-64-10 in a few thousand batches.
2. Improve initialization and check that the network learns much faster
   and reaches over 97% test accuracy. A good basic initialization scheme is so-called Glorot initialization. For a set of weights going from a layer with $n_{in}$ neurons to a layer with $n_{out}$ neurons, it samples each weight from normal distribution with $0$ mean and standard deviation of $\sqrt{\frac{2}{n_{in}+n_{out}}}$.
3. Check, that with proper initialization we can train architecture
   input-64-64-64-64-64-10, while with bad initialization it does
   not even get off the ground.
4. Add dropout implemented in pytorch
5. Check that with 10 hidden layers (64 units each) even with proper
    initialization the network has a hard time to start learning.
6. Implement batch normalization (use train mode also for testing - it should perform well enough):
    * compute batch mean and variance
    * add new variables beta and gamma
    * check that the networks learns much faster for 5 layers
    * check that the network learns even for 10 hidden layers.
7. So far we worked with a fully connected network. Design and implement in pytorch (by using pytorch functions) a simple convolutional network and achieve 99% test accuracy. The architecture is up to you, but even a few convolutional layers should be enough.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn import init
import torchvision
import torchvision.transforms as transforms

In [2]:
class Linear(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        self.bias = Parameter(torch.Tensor(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        init.xavier_normal_(self.weight)
        init.zeros_(self.bias)

    def forward(self, x):
        r = x.matmul(self.weight.t())
        r += self.bias
        return r

In [3]:
def batch_norm(x, gamma, beta, moving_mean, moving_var, eps=1e-5, momentum=0.9):
    if not torch.is_grad_enabled():
        x_norm = (x-moving_mean) / torch.sqrt(moving_var+eps)
    else:
        mean = x.mean(dim=0, keepdims=True)
        var = ((x-mean) ** 2).mean(dim=0, keepdims=True)
        x_norm = (x-mean) / torch.sqrt(var+eps)
        moving_mean = momentum * moving_mean + (1-momentum) * mean
        moving_var = momentum * moving_var + (1-momentum) * var
    
    x = gamma * x_norm + beta
    return x, moving_mean.data, moving_var.data

class BatchNorm(nn.Module):
    def __init__(self):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(1,64))
        self.beta = nn.Parameter(torch.zeros(1,64))
        self.moving_mean = torch.zeros(1,64)
        self.moving_var = torch.ones(1,64)

    def forward(self, x, eps=1e-5, momentum=0.9):
        if self.moving_mean.device != x.device:
            self.moving_mean = self.moving_mean.to(x.device)
            self.moving_var = self.moving_var.to(x.device)
        
        x, self.moving_mean, self.moving_var = batch_norm(x, self.gamma, self.beta, self.moving_mean, 
                                                          self.moving_var, eps, momentum)
        
        return x

In [4]:
class Net(nn.Module):
    def __init__(self, n_hidden_layers=4, dropout_prob=0.1, use_dropout=False):
        super(Net, self).__init__()

        self.n_hidden_layers = n_hidden_layers
        self.dropout_prob = dropout_prob
        self.use_dropout = use_dropout

        self.input_layer = Linear(784, 64)
        self.hidden_layers = [Linear(64, 64) for _ in range(n_hidden_layers)]
        self.output_layer = Linear(64, 10)
        self.dropout = nn.Dropout(dropout_prob)
        self.batch_norm = [BatchNorm() for _ in range(n_hidden_layers)]

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.input_layer(x))

        for i in range(self.n_hidden_layers):
            x = self.hidden_layers[i](x)
            x = self.batch_norm[i](x)
            x = F.relu(x)
            if self.use_dropout:
                x = self.dropout(x)

        x = self.output_layer(x)
        return x

In [5]:
class MnistTrainer(object):
    def __init__(self, batch_size):
        transform = transforms.Compose(
                [transforms.ToTensor()])
        self.trainset = torchvision.datasets.MNIST(
            root='./data',
            download=True,
            train=True,
            transform=transform)
        self.trainloader = torch.utils.data.DataLoader(
            self.trainset, batch_size=batch_size, shuffle=True, num_workers=2)

        self.testset = torchvision.datasets.MNIST(
            root='./data',
            train=False,
            download=True, transform=transform)
        self.testloader = torch.utils.data.DataLoader(
            self.testset, batch_size=1, shuffle=False, num_workers=2)

    def train(self):
        net = Net()

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=0.05, momentum=0.9)

        for epoch in range(20):
            running_loss = 0.0
            for i, data in enumerate(self.trainloader, 0):
                inputs, labels = data
                optimizer.zero_grad()

                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                if i % 100 == 99:
                    print('[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 100))
                    running_loss = 0.0
            correct = 0
            total = 0
            with torch.no_grad():
                for data in self.testloader:
                    images, labels = data
                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            print('Accuracy of the network on the {} test images: {} %'.format(
                total, 100 * correct / total))

In [6]:
trainer = MnistTrainer(batch_size=128)
trainer.train()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

[1,   100] loss: 0.765
[1,   200] loss: 0.342
[1,   300] loss: 0.296
[1,   400] loss: 0.260
Accuracy of the network on the 10000 test images: 94.02 %
[2,   100] loss: 0.208
[2,   200] loss: 0.209
[2,   300] loss: 0.209
[2,   400] loss: 0.206
Accuracy of the network on the 10000 test images: 94.55 %
[3,   100] loss: 0.172
[3,   200] loss: 0.173
[3,   300] loss: 0.176
[3,   400] loss: 0.170
Accuracy of the network on the 10000 test images: 94.95 %
[4,   100] loss: 0.146
[4,   200] loss: 0.156
[4,   300] loss: 0.156
[4,   400] loss: 0.156
Accuracy of the network on the 10000 test images: 95.29 %
[5,   100] loss: 0.129
[5,   200] loss: 0.146
[5,   300] loss: 0.139
[5,   400] loss: 0.145
Accuracy of the network on the 10000 test images: 95.15 %
[6,   100] loss: 0.122
[6,   200] loss: 0.135
[6,   300] loss: 0.131
[6,   400] loss: 0.132
Accuracy of the network on the 10000 test images: 95.65 %
[7,   100] loss: 0.116
[7

In [7]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 5, 1, 2)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 32, 5, 1, 2)
        self.fc1 = nn.Linear(32*7*7, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x


In [8]:
class ConvMnistTrainer(object):
    def __init__(self, batch_size):
        transform = transforms.Compose(
                [transforms.ToTensor()])
        self.trainset = torchvision.datasets.MNIST(
            root='./data',
            download=True,
            train=True,
            transform=transform)
        self.trainloader = torch.utils.data.DataLoader(
            self.trainset, batch_size=batch_size, shuffle=True, num_workers=2)

        self.testset = torchvision.datasets.MNIST(
            root='./data',
            train=False,
            download=True, transform=transform)
        self.testloader = torch.utils.data.DataLoader(
            self.testset, batch_size=1, shuffle=False, num_workers=2)

    def train(self):
        net = ConvNet()

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=0.05, momentum=0.9)

        for epoch in range(20):
            running_loss = 0.0
            for i, data in enumerate(self.trainloader, 0):
                inputs, labels = data
                optimizer.zero_grad()

                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                if i % 100 == 99:
                    print('[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 100))
                    running_loss = 0.0
            correct = 0
            total = 0
            with torch.no_grad():
                for data in self.testloader:
                    images, labels = data
                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            print('Accuracy of the network on the {} test images: {} %'.format(
                total, 100 * correct / total))

In [9]:
trainer = ConvMnistTrainer(batch_size=128)
trainer.train()

[1,   100] loss: 0.690
[1,   200] loss: 0.119
[1,   300] loss: 0.083
[1,   400] loss: 0.068
Accuracy of the network on the 10000 test images: 98.68 %
[2,   100] loss: 0.052
[2,   200] loss: 0.049
[2,   300] loss: 0.050
[2,   400] loss: 0.047
Accuracy of the network on the 10000 test images: 98.49 %
[3,   100] loss: 0.038
[3,   200] loss: 0.036
[3,   300] loss: 0.035
[3,   400] loss: 0.037
Accuracy of the network on the 10000 test images: 98.99 %
[4,   100] loss: 0.028
[4,   200] loss: 0.026
[4,   300] loss: 0.030
[4,   400] loss: 0.030
Accuracy of the network on the 10000 test images: 98.79 %
[5,   100] loss: 0.024
[5,   200] loss: 0.021
[5,   300] loss: 0.023
[5,   400] loss: 0.022
Accuracy of the network on the 10000 test images: 98.76 %
[6,   100] loss: 0.019
[6,   200] loss: 0.023
[6,   300] loss: 0.020
[6,   400] loss: 0.020
Accuracy of the network on the 10000 test images: 99.07 %
[7,   100] loss: 0.014
[7,   200] loss: 0.019
[7,   300] loss: 0.024
[7,   400] loss: 0.019
Accuracy